In [1]:
print("Starting the script...")

Starting the script...


In [2]:
import pandas as pd
import glob
import os

In [34]:
from dotenv import load_dotenv
from openai import OpenAI

env_path = '/home/chb3333/yulab/chb3333/gem-patho/data_extraction/cancertype_location_description/OpenAI_key.env'
load_dotenv(env_path)

True

In [4]:
# ----- Part 1: Process the TCGA sample sheet -----

In [11]:
print("Loading the cleaned TCGA samples without ICD data...")
sample_sheet_path = "/home/chb3333/yulab/chb3333/gem-patho/data_extraction/cancertype_location_description/location_desciption_withoutICD/patients_missing_icd.parquet"
sample_df = pd.read_parquet(sample_sheet_path)
print("Sample sheet loaded. Total rows:", len(sample_df))

Loading the cleaned TCGA samples without ICD data...
Sample sheet loaded. Total rows: 204


In [12]:
sample_df

,Project ID,Case ID,Sample ID,Sample Type,Normalized Sample Type,Study Abbreviation,Study Name
69,TCGA-GBM,TCGA-06-6391,"TCGA-06-6391-10A, TCGA-06-6391-01A","Blood Derived Normal, Primary Tumor","Blood Derived Normal, Primary Tumor",GBM,Glioblastoma multiforme
144,TCGA-KICH,TCGA-KN-8430,"TCGA-KN-8430-11A, TCGA-KN-8430-01A","Solid Tissue Normal, Primary Tumor","Primary Tumor, Solid Tissue Normal",KICH,Kidney Chromophobe
431,TCGA-KIRC,TCGA-BP-4986,"TCGA-BP-4986-01A, TCGA-BP-4986-11A","Primary Tumor, Solid Tissue Normal","Primary Tumor, Solid Tissue Normal",KIRC,Kidney renal clear cell carcinoma
435,TCGA-KIRC,TCGA-CZ-4865,"TCGA-CZ-4865-01A, TCGA-CZ-4865-11A","Primary Tumor, Solid Tissue Normal","Primary Tumor, Solid Tissue Normal",KIRC,Kidney renal clear cell carcinoma
440,TCGA-KIRC,TCGA-BP-4988,"TCGA-BP-4988-01A, TCGA-BP-4988-11A","Primary Tumor, Solid Tissue Normal","Primary Tumor, Solid Tissue Normal",KIRC,Kidney renal clear cell carcinoma
...,...,...,...,...,...,...,...
10483,TCGA-UCEC,TCGA-EY-A1G7,"TCGA-EY-A1G7-10A, TCGA-EY-A1G7-01A","Blood Derived Normal, Primary Tumor","Blood Derived Normal, Primary Tumor",UCEC,Uterine Corpus Endometrial Carcinoma
10484,TCGA-UCEC,TCGA-EY-A1G8,"TCGA-EY-A1G8-10A, TCGA-EY-A1G8-01A","Blood Derived Normal, Primary Tumor","Blood Derived Normal, Primary Tumor",UCEC,Uterine Corpus Endometrial Carcinoma
10486,TCGA-UCEC,TCGA-AX-A2H8,"TCGA-AX-A2H8-01A, TCGA-AX-A2H8-11A","Primary Tumor, Solid Tissue Normal","Primary Tumor, Solid Tissue Normal",UCEC,Uterine Corpus Endometrial Carcinoma
10514,TCGA-UCEC,TCGA-BG-A221,"TCGA-BG-A221-10A, TCGA-BG-A221-01A","Blood Derived Normal, Primary Tumor","Blood Derived Normal, Primary Tumor",UCEC,Uterine Corpus Endometrial Carcinoma


In [18]:
result_df = sample_df.drop(['Project ID', "Study Abbreviation", "Sample ID", "Sample Type"], axis=1)

In [19]:
result_df

,Case ID,Normalized Sample Type,Study Name
69,TCGA-06-6391,"Blood Derived Normal, Primary Tumor",Glioblastoma multiforme
144,TCGA-KN-8430,"Primary Tumor, Solid Tissue Normal",Kidney Chromophobe
431,TCGA-BP-4986,"Primary Tumor, Solid Tissue Normal",Kidney renal clear cell carcinoma
435,TCGA-CZ-4865,"Primary Tumor, Solid Tissue Normal",Kidney renal clear cell carcinoma
440,TCGA-BP-4988,"Primary Tumor, Solid Tissue Normal",Kidney renal clear cell carcinoma
...,...,...,...
10483,TCGA-EY-A1G7,"Blood Derived Normal, Primary Tumor",Uterine Corpus Endometrial Carcinoma
10484,TCGA-EY-A1G8,"Blood Derived Normal, Primary Tumor",Uterine Corpus Endometrial Carcinoma
10486,TCGA-AX-A2H8,"Primary Tumor, Solid Tissue Normal",Uterine Corpus Endometrial Carcinoma
10514,TCGA-BG-A221,"Blood Derived Normal, Primary Tumor",Uterine Corpus Endometrial Carcinoma


In [20]:
group_cols = ['Normalized Sample Type', 'Study Name']

# Group by these columns and aggregate the Case IDs into a list
grouped_df = result_df.groupby(group_cols, dropna=False)['Case ID'].apply(list).reset_index()


# If you prefer a comma-separated string instead of a list, you can convert it as follows:
grouped_df['Case IDs'] = grouped_df['Case ID'].apply(lambda ids: ', '.join(ids))

# Drop the original 'Case ID' column (if only the aggregated version is needed)
grouped_df = grouped_df.drop(columns=['Case ID'])

In [21]:
grouped_df

,Normalized Sample Type,Study Name,Case IDs
0,"Blood Derived Normal, Metastatic",Skin Cutaneous Melanoma,TCGA-WE-A8ZR
1,"Blood Derived Normal, Metastatic",Thyroid carcinoma,TCGA-EM-A2CS
2,"Blood Derived Normal, Primary Tumor",Adrenocortical carcinoma,TCGA-P6-A5OG
3,"Blood Derived Normal, Primary Tumor",Bladder Urothelial Carcinoma,TCGA-E7-A8O8
4,"Blood Derived Normal, Primary Tumor",Brain Lower Grade Glioma,"TCGA-HT-A619, TCGA-R8-A73M"
5,"Blood Derived Normal, Primary Tumor",Breast invasive carcinoma,"TCGA-AR-A2LR, TCGA-D8-A146, TCGA-BH-A0B6, TCGA..."
6,"Blood Derived Normal, Primary Tumor",Cervical squamous cell carcinoma and endocervi...,"TCGA-C5-A902, TCGA-VS-A8EK, TCGA-C5-A901, TCGA..."
7,"Blood Derived Normal, Primary Tumor",Cholangiocarcinoma,"TCGA-5A-A8ZF, TCGA-4G-AAZG, TCGA-W5-AA2J, TCGA..."
8,"Blood Derived Normal, Primary Tumor",Colon adenocarcinoma,"TCGA-G4-6314, TCGA-AA-3815, TCGA-AA-3967, TCGA..."
9,"Blood Derived Normal, Primary Tumor",Esophageal carcinoma,TCGA-IC-A6RF


In [30]:
chatgpt_query_info = grouped_df
chatgpt_query_info = chatgpt_query_info.rename(columns={"Study Name": "CANCER_TYPE_NAME"})
chatgpt_query_info = chatgpt_query_info.rename(columns={"Normalized Sample Type": "Normalized_Sample_Type"})

In [31]:
# ChatGPT
# Here we just reaplce the fine grained information with Cancer Type Nme

In [32]:
# Define the prompt template
template = """
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {{
      "role": "system",
      "content": [
        {{
          "type": "text",
          "text": "You are an expert oncologist specializing in cancer survival. You are explaining complex oncological and genomic factors to another expert in a structured, precise manner. Focus on step-by-step reasoning, integrating knowledge of tumor origin, histology, genetic mutations, and sampling biases. Avoid redundant explanations and ensure biological clarity while emphasizing survival impact."
        }}
      ]
    }},
    {{
      "role": "user",
      "content": [
        {{
          "type": "text",
          "text": "Describe how {CANCER_TYPE_NAME} impacts survival, focusing on:\n- Tumor Origin: How {CANCER_TYPE_NAME} influences metastatic patterns, patient survival, and treatment accessibility.\n- Histology: How {CANCER_TYPE_NAME} interacts with mutation profiles to drive outcomes.\n- Sampling Bias: Limitations of {Normalized_Sample_Type} in genomic analysis.\n- Key Genes: Identify 5-8 survival-associated genes, explaining their biological mechanisms (e.g., proliferation, immune response, apoptosis regulation).\n\nFor each aspect, explain step by step:\n- How it influences survival (positive/negative)\n- Biological rationale without excessive jargon\n- Potential biases in genomic interpretation\n\nConclude with a synthesis of key survival risk factors, emphasizing clinically relevant insights and their impact on patient survival."
        }}
      ]
    }}
  ],
  response_format={{ "type": "text" }},
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
"""

In [35]:
# Build the list of prompts and store the corresponding Case IDs
prompts = []
case_ids_list = []

for index, row in chatgpt_query_info.iterrows():
    prompt_filled = template.format(
        CANCER_TYPE_NAME=row["CANCER_TYPE_NAME"],
        Normalized_Sample_Type=row["Normalized_Sample_Type"]
    )
    prompts.append(prompt_filled)
    case_ids_list.append(row["Case IDs"])

print("First prompt example:")
print(prompts[0])

# %%
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

responses = []

for idx, prompt in enumerate(prompts):
    print(f"Processing prompt {idx+1}/{len(prompts)}...")
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": [
                        {
                            "type": "text",
                            "text": "You are an expert oncologist specializing in cancer survival. You are explaining complex oncological and genomic factors to another expert in a structured, precise manner. Focus on step-by-step reasoning, integrating knowledge of tumor origin, histology, genetic mutations, and sampling biases. Avoid redundant explanations and ensure biological clarity while emphasizing survival impact."
                        }
                    ]
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt
                        }
                    ]
                }
            ],
            response_format={"type": "text"},
            temperature=1,
            max_completion_tokens=2048,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        responses.append(response)
        print("Response received.")
    except Exception as e:
        print(f"Error processing prompt {idx+1}: {e}")

First prompt example:

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "You are an expert oncologist specializing in cancer survival. You are explaining complex oncological and genomic factors to another expert in a structured, precise manner. Focus on step-by-step reasoning, integrating knowledge of tumor origin, histology, genetic mutations, and sampling biases. Avoid redundant explanations and ensure biological clarity while emphasizing survival impact."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Describe how Skin Cutaneous Melanoma impacts survival, focusing on:
- Tumor Origin: How Skin Cutaneous Melanoma influences metastatic patterns, patient survival, and treatment accessibility.
- Histology: How Skin Cutaneous Melanoma interacts with mutation profiles to drive outcomes

In [36]:
# Build a DataFrame that includes the prompt, response, and associated Case IDs
data = []
for prompt, resp, case_ids in zip(prompts, responses, case_ids_list):
    try:
        answer_text = resp.choices[0].message.content
    except Exception as e:
        answer_text = None
        print(f"Error extracting answer: {e}")
    data.append({
        "prompt": prompt,
        "response": answer_text,
        "Case IDs": case_ids
    })

In [37]:
df_responses = pd.DataFrame(data)


In [38]:
csv_path = "/home/chb3333/yulab/chb3333/gem-patho/data_extraction/cancertype_location_description/location_desciption_withoutICD/chat_responses.csv"
parquet_path = "/home/chb3333/yulab/chb3333/gem-patho/data_extraction/cancertype_location_description/location_desciption_withoutICD/chat_responses.parquet"


In [39]:
df_responses.to_csv(csv_path, index=False)
print(f"Responses saved to {csv_path}")

df_responses.to_parquet(parquet_path, index=False)
print(f"Responses saved to {parquet_path}")

Responses saved to /home/chb3333/yulab/chb3333/gem-patho/data_extraction/cancertype_location_description/location_desciption_withoutICD/chat_responses.csv
Responses saved to /home/chb3333/yulab/chb3333/gem-patho/data_extraction/cancertype_location_description/location_desciption_withoutICD/chat_responses.parquet


In [40]:
description_meta = grouped_df

if len(description_meta) != len(df_responses):
    print("Warning: The number of rows differ!")
else:
    print("Row counts match. Proceeding with validation...")

Row counts match. Proceeding with validation...


In [41]:
description_meta["generated_description"] = df_responses["response"].values


In [42]:
description_meta

,Normalized Sample Type,Study Name,Case IDs,generated_description
0,"Blood Derived Normal, Metastatic",Skin Cutaneous Melanoma,TCGA-WE-A8ZR,## Tumor Origin and Its Influence on Survival\...
1,"Blood Derived Normal, Metastatic",Thyroid carcinoma,TCGA-EM-A2CS,Thyroid carcinoma showcases distinct influence...
2,"Blood Derived Normal, Primary Tumor",Adrenocortical carcinoma,TCGA-P6-A5OG,### Tumor Origin\n\n**Influence on Survival:**...
3,"Blood Derived Normal, Primary Tumor",Bladder Urothelial Carcinoma,TCGA-E7-A8O8,### Tumor Origin and Metastatic Patterns\n\n**...
4,"Blood Derived Normal, Primary Tumor",Brain Lower Grade Glioma,"TCGA-HT-A619, TCGA-R8-A73M",### Tumor Origin\n\n**Influence on Survival:**...
5,"Blood Derived Normal, Primary Tumor",Breast invasive carcinoma,"TCGA-AR-A2LR, TCGA-D8-A146, TCGA-BH-A0B6, TCGA...",### Tumor Origin and Its Influence on Survival...
6,"Blood Derived Normal, Primary Tumor",Cervical squamous cell carcinoma and endocervi...,"TCGA-C5-A902, TCGA-VS-A8EK, TCGA-C5-A901, TCGA...","Sure, let's analyze the impact of cervical squ..."
7,"Blood Derived Normal, Primary Tumor",Cholangiocarcinoma,"TCGA-5A-A8ZF, TCGA-4G-AAZG, TCGA-W5-AA2J, TCGA...",### Tumor Origin\n\n**Influence on Survival:**...
8,"Blood Derived Normal, Primary Tumor",Colon adenocarcinoma,"TCGA-G4-6314, TCGA-AA-3815, TCGA-AA-3967, TCGA...",**Tumor Origin and Metastatic Patterns:**\n\n1...
9,"Blood Derived Normal, Primary Tumor",Esophageal carcinoma,TCGA-IC-A6RF,"Certainly, let's delve into the complexities o..."


In [ ]:
description_meta.to_parquet(
    "/home/chb3333/yulab/chb3333/gem-patho/data_extraction/cancertype_location_description/location_desciption_withoutICD/description_meta_with_answers.parquet",
    index=False
)
description_meta.to_parquet(
    "/home/chb3333/yulab/chb3333/gem-patho/data_extraction/cancertype_location_description/location_desciption_withoutICD/description_meta.parquet",
    index=False
)
print("Updated description_meta saved successfully.")

OSError: Cannot save file into a non-existent directory: '/home/chb3333/yulab/chb3333/gem-patho/data_extraction/cancertype_location_description/location_description_withoutICD'